In [1]:
import requests 
from tqdm import tqdm 
import pandas as pd 
import numpy as np
import multiprocessing 
import time 


from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver 
from bs4 import BeautifulSoup 

In [2]:
df = pd.read_csv('celeb(1).csv')
df.head()

,celeb_id,celeb_name,profession,famous_for,img_link,profile_link
0,nm0000079,Raquel Welch,Actress,The Three Musketeers,https://m.media-amazon.com/images/M/MV5BZDJhMz...,https://www.imdb.com/name/nm0000079
1,nm0050959,Pedro Pascal,Actor,The Mandalorian,https://m.media-amazon.com/images/M/MV5BMGNlYT...,https://www.imdb.com/name/nm0050959
2,nm8165602,Bella Ramsey,Actress,Game of Thrones,https://m.media-amazon.com/images/M/MV5BMWJkZW...,https://www.imdb.com/name/nm8165602
3,nm0001491,Melanie Lynskey,Actress,Togetherness,https://m.media-amazon.com/images/M/MV5BMjIzND...,https://www.imdb.com/name/nm0001491
4,nm8567338,Sasha Calle,Actress,The Flash,https://m.media-amazon.com/images/M/MV5BZTc1Ym...,https://www.imdb.com/name/nm8567338


In [34]:
browser = webdriver.Chrome(ChromeDriverManager().install())
link = 'https://www.imdb.com'
browser.get(link)
soup = BeautifulSoup(browser.page_source,'html.parser')

/tmp/ipykernel_4261/346165053.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# multiprocessing the data 

In [4]:
def details(df,start,end):
    data = []
    for link in tqdm(df['profile_link'][start:end]):
        browser.get(link)
        time.sleep(1)
        soup = BeautifulSoup(browser.page_source,'html.parser')

        # profession 
        try:
            prof        = ','.join([prof.text for prof in soup.find('ul',class_ = 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt').find_all('li')])
        except:
            prof        = np.nan

        # work done or famous for 
        try:
            famous_for  = ','.join([fame.text for fame in soup.find('div',class_ = 'ipc-sub-grid ipc-sub-grid--page-span-2 ipc-sub-grid--wraps-at-above-l ipc-shoveler__grid').find_all('div',class_ = 'ipc-primary-image-list-card__content-top')])
        except:
            famous_for  = np.nan

        # awards 
        try:    
            awards      = soup.find('label',class_ ='ipc-metadata-list-item__list-content-item').text
        except:
            awards      = np.nan

        # Birthday 
        try:
            b_day       = soup.find('aside',class_ ='sc-dec7a8b-0 zApYU sc-b0d57fa9-5 efcXeK').find('div',class_ = 'sc-dec7a8b-1 kvzEHe').find_all('span')[1].text

        except:
            b_day       = np.nan

        # height, place of birth , name of the parents 
        for i in soup.find('ul',class_='ipc-metadata-list ipc-metadata-list--dividers-all ipc-metadata-list--base').find_all('li'):
            try:
                if ('Born' in i.text):
                    b_place = (i.find('div').find_all('li')[1].text)
                elif ('Height' in i.text):
                    height  = (i.find('ul').find('li').text[7:-3])
                elif ('Parents' in i.text):
                    parent = (i.find('div').text)
            except:
                b_place = np.nan
                height = np.nan
                parent = np.nan

        data.append([prof,awards,b_day,famous_for,b_place,height,parent])
    df_ = pd.DataFrame(data,columns= ['prof','awards','b_day','famous_for','b_place','height','parent'])
    df_.to_csv('profile_details.csv',index = False)

In [5]:
df_= pd.read_csv('profile_details.csv')
df_ 

,prof,awards,b_day,famous_for,b_place,height,parent
0,"Actress,Producer,Director",1 win,"February 6, 1982","Before We Go,Star Trek Into Darkness,She's Out...","London, England, UK",(1.66,Trevor Eve
1,"Actor,Producer,Soundtrack",26 wins & 30 nominations total,"April 10, 1988","The Sixth Sense,Artificial Intelligence: AI,Pa...","Los Angeles, California, USA",1.68,Theresa Seifert
2,"Actress,Producer,Director",17 wins & 37 nominations,"February 23, 1994","Planetary War,I Am Sam,Man on Fire,Bees' Secre...","Conyers, Georgia, USA",1.63,Joy Fanning
3,"Actress,Director,Writer",6 nominations,"October 10, 1988","The Lovely Bones,iZombie,A Christmas Prince,Th...","Auckland, New Zealand",(1.62,"John George Whitfield ""Mac"" McIver"
4,"Actor,Producer,Soundtrack",29 wins & 29 nominations,"January 13, 1977",The Lord of the Rings: The Fellowship of the R...,"Canterbury, Kent, England, UK",(1.80,Sonia Constance Josephine Copeland
...,...,...,...,...,...,...,...
95,"Actress,Producer,Soundtrack",4 wins & 14 nominations,"March 10, 1992","Hannah Montana: The Movie,Spy Kids 2: Island o...","Los Angeles, California, USA",1.60,Eugene Osment
96,"Actor,Writer,Producer",5 wins & 23 nominations total,"January 28, 1993","We're the Millers,The Maze Runner,Detroit,The ...","England, UK",(1.91,Caroline Barrah
97,"Actress,Producer,Additional Crew",12 wins & 46 nominations total,"March 11, 1993","The Last Duel,Free Guy,Killing Eve,Star Wars: ...","Liverpool, Merseyside, England, UK",1.73,Donna Comer
98,"Actress,Director,Producer",8 wins & 27 nominations,"October 8, 1997","The Duff,Midnight Sun,Blended,Scream: The TV S...","Pembroke Pines, Florida, USA",1.73,Tamara Thorne (Beckett)


In [35]:
link = 'https://www.imdb.com/name/nm0000001'
browser.get(link)

In [36]:
def details(df,start,end):
    data = []

    for i in tqdm(df['celeb_id'][start:end]):

        link = 'https://www.imdb.com/name/' + i 
        browser.get(link)
        time.sleep(2)
        soup = BeautifulSoup(browser.page_source,'html.parser')

        try:
            prof        = ','.join([prof.text for prof in soup.find('ul',class_ = 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt').find_all('li')])
        except:
            prof        = np.nan

        # work done or famous for 
        try:
            famous_for  = ','.join([fame.text for fame in soup.find('div',class_ = 'ipc-sub-grid ipc-sub-grid--page-span-2 ipc-sub-grid--wraps-at-above-l ipc-shoveler__grid').find_all('div',class_ = 'ipc-primary-image-list-card__content-top')])
        except:
            famous_for  = np.nan

        # awards 
        try:    
            awards      = soup.find('label',class_ ='ipc-metadata-list-item__list-content-item').text
        except:
            awards      = np.nan

        # Birthday 
        try:
            b_day       = soup.find('aside',class_ ='sc-dec7a8b-0 zApYU sc-b0d57fa9-5 efcXeK').find('div',class_ = 'sc-dec7a8b-1 kvzEHe').find_all('span')[1].text

        except:
            b_day       = np.nan

        # height, place of birth , name of the parents 
        for i in soup.find('ul',class_='ipc-metadata-list ipc-metadata-list--dividers-all ipc-metadata-list--base').find_all('li'):
            try:
                if ('Born' in i.text):
                    b_place    =    i.find('div').find_all('li')[1].text
                elif ('Height' in i.text):
                    height     =    i.find('ul').find('li').text[7:-3]
                elif ('Parents' in i.text):
                    parent     =    i.find('div').text
            except:
                b_place        =    np.nan
                height         =    np.nan
                parent         =    np.nan

        data.append([prof,awards,b_day,famous_for,b_place,height,parent])
    # df_ = pd.DataFrame(data,columns= ['prof','awards','b_day','famous_for','b_place','height','parent'])
    # df_.to_csv(f'{str(i)}.csv',index = False)

In [30]:
details(df,1,101)

  0%|                                                     | 0/5 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [28]:
p1 = multiprocessing.Process(target = details,args = (df,1,101))
p2 = multiprocessing.Process(target = details,args = (df,101,201))
p3 = multiprocessing.Process(target = details,args = (df,201,301))
p4 = multiprocessing.Process(target = details,args = (df,301,401))
p5 = multiprocessing.Process(target = details,args = (df,401,501))

# start of multiple process
p1.start()
p2.start()
p3.start()
p4.start()
p5.start()

# joining of the processess
p1.join()
p2.join()
p3.join()
p4.join()
p5.join()


  0%|                                                     | 0/5 [00:20<?, ?it/s]
Process Process-7:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_4261/3639539385.py", line 9, in details
    soup = BeautifulSoup(browser.page_source,'html.parser')
  File "/usr/lib/python3/dist-packages/bs4/__init__.py", line 312, in __init__
    elif len(markup) <= 256 and (
TypeError: object of type 'NoneType' has no len()
  0%|                                                     | 0/5 [00:31<?, ?it/s]
Process Process-10:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tm

In [3]:
df_ = pd.DataFrame(data,columns= ['prof','awards','b_day','famous_for','b_place','height','parent'])
df_.to_csv('profile_details.csv',index = False)

NameError: name 'data' is not defined

In [7]:
df = pd.read_csv('profile_details.csv')
df.head()

,prof,awards,b_day,famous_for,b_place,height,parent
0,"Actress,Producer,Director",1 win,"February 6, 1982","Before We Go,Star Trek Into Darkness,She's Out...","London, England, UK",(1.66,Trevor Eve
1,"Actor,Producer,Soundtrack",26 wins & 30 nominations total,"April 10, 1988","The Sixth Sense,Artificial Intelligence: AI,Pa...","Los Angeles, California, USA",1.68,Theresa Seifert
2,"Actress,Producer,Director",17 wins & 37 nominations,"February 23, 1994","Planetary War,I Am Sam,Man on Fire,Bees' Secre...","Conyers, Georgia, USA",1.63,Joy Fanning
3,"Actress,Director,Writer",6 nominations,"October 10, 1988","The Lovely Bones,iZombie,A Christmas Prince,Th...","Auckland, New Zealand",(1.62,"John George Whitfield ""Mac"" McIver"
4,"Actor,Producer,Soundtrack",29 wins & 29 nominations,"January 13, 1977",The Lord of the Rings: The Fellowship of the R...,"Canterbury, Kent, England, UK",(1.80,Sonia Constance Josephine Copeland


In [5]:
df_ = pd.DataFrame(data,columns= ['prof','awards','b_day','famous_for','b_place','height','parent'])

NameError: name 'data' is not defined

In [ ]:
master_data = pd.concat([df[:5],df_],axis = 1)

In [42]:
def details(df,start,end):
    data = []
    for i in tqdm(df['celeb_id'][start:end]):
        s = HTMLSession()
        link = 'https://www.imdb.com/name/'+i
        r = s.get(link)
        soup = BeautifulSoup(r.text,'html.parser')
        
        # professsion 
        try:
            prof        = ','.join([prof.text for prof in soup.find('ul',class_ = 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt').find_all('li')])
        except:
            prof        = np.nan

        # work done or famous for 
        try:
            famous_for  = ','.join([fame.text for fame in soup.find('div',class_ = 'ipc-sub-grid ipc-sub-grid--page-span-2 ipc-sub-grid--wraps-at-above-l ipc-shoveler__grid').find_all('div',class_ = 'ipc-primary-image-list-card__content-top')])
        except:
            famous_for  = np.nan

        # awards 
        try:    
            awards      = soup.find('label',class_ ='ipc-metadata-list-item__list-content-item').text
        except:
            awards      = np.nan

        # Birthday 
        try:
            b_day       = soup.find('aside',class_ ='sc-dec7a8b-0 zApYU sc-b0d57fa9-5 efcXeK').find('div',class_ = 'sc-dec7a8b-1 kvzEHe').find_all('span')[1].text

        except:
            b_day       = np.nan

        # height, place of birth , name of the parents 
        for i in soup.find('ul',class_='ipc-metadata-list ipc-metadata-list--dividers-all ipc-metadata-list--base').find_all('li'):
            try:
                if ('Born' in i.text):
                    b_place = (i.find('div').find_all('li')[1].text)
                elif ('Height' in i.text):
                    height  = (i.find('ul').find('li').text[7:-3])
                elif ('Parents' in i.text):
                    parent = (i.find('div').text)
            except:
                b_place = np.nan
                height = np.nan
                parent = np.nan

        data.append([prof,awards,b_day,famous_for,b_place,height,parent])
    return data
#     df_ = pd.DataFrame(data,columns= ['prof','awards','b_day','famous_for','b_place','height','parent'])
#     df_.to_csv('profile_details.csv',index = False)

In [44]:
print(details(df,1,101))

100%|█████████████████████████████████████████| 100/100 [13:47<00:00,  8.27s/it]

[['Actor,Producer,Soundtrack', '7 wins & 15 nominations', 'April 2, 1975', 'The Mandalorian,Kingsman: The Golden Circle,Game of Thrones,Wonder Woman 1984', 'Santiago de Chile, Metropolitan Region, Chile', ' (1.79', 'Veronica Pascal'], ['Actress,Soundtrack,Writer', '2 wins & 8 nominations', 'September 30, 2003', 'Game of Thrones,The Last of Us,Catherine Called Birdy,Hilda', 'Nottinghamshire, England, UK', '(1.56', 'Alex Ramsey'], ['Actress,Soundtrack', '15 wins & 32 nominations total', 'May 16, 1977', 'Togetherness,Heavenly Creatures,Up in the Air,The Intervention', 'New Plymouth, New Zealand', '1.70', 'Kay Lynskey'], ['Actress,Music Department', '3 nominations', 'August 7, 1995', 'The Flash,Supergirl,The Young and the Restless', 'Boston, Massachusetts, USA', '1.70', 'Samira Calle'], ['Actress', '1 win & 2 nominations', 'March 28, 1977', 'The Last of Us,24,Runaways,Revolution', 'St. Louis, Missouri, USA', '1.70', 'Sandy Wersching'], ['Actress,Writer,Soundtrack', 'Luke Ritchie', 'August 